In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import random
from itertools import product
import torch

In [ ]:
!pip install -U pytorch-forecasting

In [ ]:
!pip install neuralforecast

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.8/285.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.6 MB/s eta 0:00:00


# **Read Data**

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Train Data:
# Store - the store number
# Dept - the department number
# Date - the week
# Weekly_Sales -  sales for the given department in the given store
# IsHoliday - whether the week is a special holiday week
import pandas as pd

train_df = pd.read_csv('/content/drive/MyDrive/Walmart_Recruiting/Data/train.csv.zip')
train_df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   Weekly_Sales  421570 non-null  float64
 4   IsHoliday     421570 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 13.3+ MB


In [ ]:
# Test Data:
# Store - the store number
# Dept - the department number
# Date - the week
# IsHoliday - whether the week is a special holiday week

test_df = pd.read_csv('/content/drive/MyDrive/Walmart_Recruiting/Data/test.csv.zip')
test_df.head()

,Store,Dept,Date,IsHoliday
0,1,1,2012-11-02,False
1,1,1,2012-11-09,False
2,1,1,2012-11-16,False
3,1,1,2012-11-23,True
4,1,1,2012-11-30,False


In [ ]:
# Info about stores
stores_df = pd.read_csv('/content/drive/MyDrive/Walmart_Recruiting/Data/stores.csv')
stores_df.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [ ]:
stores_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Store   45 non-null     int64 
 1   Type    45 non-null     object
 2   Size    45 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.2+ KB


In [ ]:
# Features data:
# Store - the store number
# Date - the week
# Temperature - average temperature in the region
# Fuel_Price - cost of fuel in the region
# MarkDown1-5 - anonymized data related to promotional markdowns that Walmart is running. MarkDown data is only available after Nov 2011, and is not available for all stores all the time. Any missing value is marked with an NA.
# CPI - the consumer price index
# Unemployment - the unemployment rate
# IsHoliday - whether the week is a special holiday week

features_df = pd.read_csv('/content/drive/MyDrive/Walmart_Recruiting/Data/features.csv.zip')
features_df.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [ ]:
features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         8190 non-null   int64  
 1   Date          8190 non-null   object 
 2   Temperature   8190 non-null   float64
 3   Fuel_Price    8190 non-null   float64
 4   MarkDown1     4032 non-null   float64
 5   MarkDown2     2921 non-null   float64
 6   MarkDown3     3613 non-null   float64
 7   MarkDown4     3464 non-null   float64
 8   MarkDown5     4050 non-null   float64
 9   CPI           7605 non-null   float64
 10  Unemployment  7605 non-null   float64
 11  IsHoliday     8190 non-null   bool   
dtypes: bool(1), float64(9), int64(1), object(1)
memory usage: 712.0+ KB


# **Processing**

In [ ]:
class WalmartTFTPreprocessor:
    def __init__(self):
        self.label_encoders = {}
        self.static_features = ['Store', 'Dept', 'Type']
        self.time_known_features = ['IsHoliday', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment'] + \
                                   ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
        self.generated_time_features = ['Year', 'Month', 'Week', 'DayOfWeek', 'IsMonthStart']
        self.cyclic_features = ['Month', 'Week', 'DayOfWeek']
        self.time_observed_features = ['Weekly_Sales']

    def fit(self, train_df, features_df, stores_df):
        # Merge and prepare full training data
        full_df = train_df.merge(features_df, how='left', on=['Store', 'Date', 'IsHoliday'])
        full_df = full_df.merge(stores_df, how='left', on='Store')
        full_df['Date'] = pd.to_datetime(full_df['Date'])

        # Fit label encoders
        for col in self.static_features:
            le = LabelEncoder()
            full_df[col] = le.fit_transform(full_df[col])
            self.label_encoders[col] = le

        return self

    def transform(self, df, features_df, stores_df, is_train=True):
        df = df.copy()

        df = df.merge(features_df, how='left', on=['Store', 'Date', 'IsHoliday'])
        df = df.merge(stores_df, how='left', on='Store')
        df['Date'] = pd.to_datetime(df['Date'])
        df.sort_values(by=['Store', 'Dept', 'Date'], inplace=True)

        # Label encode static categorical variables
        for col in self.static_features:
            if col in df.columns and col in self.label_encoders:
                df[col] = self.label_encoders[col].transform(df[col])

        # Fill missing markdowns
        markdown_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
        for col in markdown_cols:
            df[f'{col}_was_missing'] = df[col].isna().astype(int)
            df[col] = df[col].fillna(0)

        # Generate time features
        df['Year'] = df['Date'].dt.year
        df['Month'] = df['Date'].dt.month
        df['Week'] = df['Date'].dt.isocalendar().week.astype(int)
        df['DayOfWeek'] = df['Date'].dt.dayofweek
        df['Day'] = df['Date'].dt.day
        df['IsMonthStart'] = df['Date'].dt.is_month_start.astype(int)

        # Cyclical encoding
        for col, period in zip(self.cyclic_features, [12, 52, 7]):
            df[f'{col}_sin'] = np.sin(2 * np.pi * df[col] / period)
            df[f'{col}_cos'] = np.cos(2 * np.pi * df[col] / period)

        # Add a unique series ID for grouping
        df['series_id'] = df['Store'].astype(str) + "_" + df['Dept'].astype(str)

        # Save original Date
        df['OriginalDate'] = df['Date']
        df.drop(columns=['Date'], inplace=True)

        df["Weekly_Sales"] = df["Weekly_Sales"].replace([np.inf, -np.inf], np.nan)
        df["Weekly_Sales"] = df.groupby("series_id")["Weekly_Sales"].transform(lambda x: x.interpolate(limit_direction="both").fillna(0))


        # Collect all available features
        features = [col for col in df.columns if col not in ['Weekly_Sales', 'OriginalDate']]

        return df, features

    def fit_transform(self, train_df, features_df, stores_df):
        self.fit(train_df, features_df, stores_df)
        return self.transform(train_df, features_df, stores_df, is_train=True)

    def get_feature_types(self):
        return {
            "static_categoricals": self.static_features,
            "time_varying_known_categoricals": ['IsHoliday'],
            "time_varying_known_reals": ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment'] +
                                        ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5'],
            "time_varying_observed_reals": ['Weekly_Sales'],
            "time_features": self.generated_time_features +
                             [f'{col}_sin' for col in self.cyclic_features] +
                             [f'{col}_cos' for col in self.cyclic_features]
        }


In [ ]:
preprocessor = WalmartTFTPreprocessor()
df, features = preprocessor.fit_transform(train_df, features_df, stores_df)

In [ ]:
import plotly.express as px
import pandas as pd

# Suppose your processed dataframe is called `df` and has these columns:
# 'OriginalDate' (datetime), 'Weekly_Sales' (float)

# Aggregate sales per date
daily_sales = df.groupby('OriginalDate')['Weekly_Sales'].sum().reset_index()

# Plot total weekly sales over time
fig = px.line(
    daily_sales,
    x='OriginalDate',
    y='Weekly_Sales',
    title='Total Weekly Sales Over Time (All Stores & Departments)',
    labels={'OriginalDate': 'Date', 'Weekly_Sales': 'Total Weekly Sales'}
)
fig.show()


In [ ]:
import plotly.express as px

# Make sure IsHoliday is int (0 or 1)
df['IsHoliday'] = df['IsHoliday'].astype(int)

fig = px.box(
    df,
    x='IsHoliday',
    y='Weekly_Sales',
    title='Sales Distribution: Holiday vs Non-Holiday Weeks',
    labels={
        'IsHoliday': 'Is Holiday (0=No, 1=Yes)',
        'Weekly_Sales': 'Weekly Sales'
    }
)

fig.show()


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import plotly.express as px

fig = px.box(
    df,
    x='Type',
    y='Weekly_Sales',
    title='Sales Distribution by Store Type',
    labels={'Type': 'Store Type', 'Weekly_Sales': 'Weekly Sales'}
)

fig.show()


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import plotly.graph_objects as go
import numpy as np

corr_cols = ['Weekly_Sales', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment'] + \
            ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']

corr = df[corr_cols].corr()

fig = go.Figure(data=go.Heatmap(
    z=corr.values,
    x=corr.columns,
    y=corr.columns,
    colorscale='RdBu',
    zmin=-1,
    zmax=1,
    colorbar=dict(title='Correlation')
))

# Add correlation values as annotations
annotations = []
for i, row in enumerate(corr.values):
    for j, val in enumerate(row):
        annotations.append(
            dict(
                x=corr.columns[j],
                y=corr.columns[i],
                text=f"{val:.2f}",
                showarrow=False,
                font=dict(color="white" if abs(val) > 0.5 else "black")
            )
        )

fig.update_layout(
    title='Feature Correlation Heatmap',
    annotations=annotations,
    xaxis_side='top',
    width=700,
    height=700
)

fig.show()


# **Training TFT model**

In [ ]:
!pip install wandb

In [ ]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tekla-tsutskha (tekla-tsutskha-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
df["time_idx"] = (df["OriginalDate"] - df["OriginalDate"].min()).dt.days

# Define forecasting horizon
max_prediction_length = 6
max_encoder_length = 30

# Feature settings
feature_types = preprocessor.get_feature_types()

In [ ]:
from pytorch_forecasting import TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer


# Convert all static categoricals to string
for col in feature_types.get("static_categoricals", []):
    df[col] = df[col].astype(str)

# Same for known categoricals (if any)
for col in feature_types.get("time_varying_known_categoricals", []):
    df[col] = df[col].astype(str)

# Define how many weeks you want to predict into the future
max_prediction_length = 6  # for example, 6 weeks

# Create a cutoff date for training vs validation
training_cutoff = df['OriginalDate'].max() - pd.Timedelta(weeks=max_prediction_length)

# Split into training and validation
df_train = df[df['OriginalDate'] <= training_cutoff].copy()
df_val = df[df['OriginalDate'] > training_cutoff].copy()

# Optional: Keep only series in val that exist in training (avoid unseen series)
df_val = df_val[df_val['series_id'].isin(df_train['series_id'].unique())]


training = TimeSeriesDataSet(
    df,
    time_idx="time_idx",
    target="Weekly_Sales",
    group_ids=["series_id"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=feature_types.get("static_categoricals", []),
    time_varying_known_categoricals=feature_types.get("time_varying_known_categoricals", []),
    time_varying_known_reals=feature_types.get("time_varying_known_reals", []),
    time_varying_unknown_reals=["Weekly_Sales"],
    target_normalizer=GroupNormalizer(groups=["series_id"]),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True,  # Add this line!
)

validation = TimeSeriesDataSet.from_dataset(training, df_val, predict=True, stop_randomization=True)



/usr/local/lib/python3.11/dist-packages/pytorch_forecasting/data/timeseries/_timeseries.py:1847: UserWarning:

Min encoder length and/or min_prediction_idx and/or min prediction length and/or lags are too large for 131 series/groups which therefore are not present in the dataset index. This means no predictions can be made for those series. First 10 removed groups: [{'__group_id__series_id': '0_62'}, {'__group_id__series_id': '0_63'}, {'__group_id__series_id': '10_46'}, {'__group_id__series_id': '10_48'}, {'__group_id__series_id': '10_62'}, {'__group_id__series_id': '10_63'}, {'__group_id__series_id': '11_62'}, {'__group_id__series_id': '11_80'}, {'__group_id__series_id': '12_41'}, {'__group_id__series_id': '12_45'}]

/usr/local/lib/python3.11/dist-packages/pytorch_forecasting/data/timeseries/_timeseries.py:1847: UserWarning:

Min encoder length and/or min_prediction_idx and/or min prediction length and/or lags are too large for 127 series/groups which therefore are not present in the 

In [ ]:
batch_size = 64

train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

In [ ]:
from pytorch_forecasting import TemporalFusionTransformer
from pytorch_forecasting.metrics import QuantileLoss

tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=16,  # smaller for quicker testing; increase for better accuracy
    dropout=0.1,
    hidden_continuous_size=8,
    output_size=7,  # number of quantiles to predict by default
    loss=QuantileLoss(),
    log_interval=10,
    reduce_on_plateau_patience=4,
)

/usr/local/lib/python3.11/dist-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.


In [ ]:
# Install NeuralForecast if you haven't yet:
# !pip install neuralforecast

import pandas as pd
from neuralforecast import NeuralForecast
from neuralforecast.models import TFT  # Temporal Fusion Transformer implemented in NeuralForecast
from sklearn.metrics import mean_absolute_error

# Prepare your DataFrame with columns:
# 'unique_id' (series id), 'ds' (datetime), 'y' (target variable)

df = df.copy()
df['unique_id'] = df['Store'].astype(str) + "_" + df['Dept'].astype(str)
df['ds'] = pd.to_datetime(df['OriginalDate'])
df['y'] = df['Weekly_Sales']

# Keep only necessary columns for NeuralForecast
df_nf = df[['unique_id', 'ds', 'y']].sort_values(['unique_id', 'ds'])

# Split train / val by date
max_date = df_nf['ds'].max()
cutoff_date = max_date - pd.Timedelta(weeks=6)

train_df = df_nf[df_nf['ds'] <= cutoff_date]
val_df = df_nf[df_nf['ds'] > cutoff_date]

wandb.init(project="walmart-tft-forecast", name="TFT-horizon39", config={
    "model": "TFT",
    "h": 39,
    "input_size": 80,
    "hidden_size": 32,
    "dropout": 0.1,
    "freq": "W-FRI",
})


# Create NeuralForecast wrapper
nf = NeuralForecast(models=[model], freq="W-FRI")

# Fit model
nf.fit(train_df)

# Predict on validation data (for the next 6 weeks per series)
preds = nf.predict()

print(preds.head())

merged = preds.merge(val_df, on=['unique_id', 'ds'], how='left')
merged = merged.dropna()

mae = mean_absolute_error(merged['y'], merged['TFT'])

# Log to wandb
wandb.log({'MAE': mae})
print(f"MAE on validation set: {mae:.2f}")

wandb.finish()


INFO:lightning_fabric.utilities.seed:Seed set to 1
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                    | Type                     | Params | Mode 
-----------------------------------------------------------------------------
0 | loss                    | MAE                      | 0      | train
1 | padder_train            | ConstantPad1d            | 0      | train
2 | scaler                  | TemporalNorm             | 0      | train
3 | embedding               | TFTEmbedding             | 128    | train
4 | temporal_encoder        | TemporalCovariateEncoder | 39.6 K | train
5 | temporal_fusion_decoder | TemporalFusionDecoder    | 16.5 K | train
6 | output_adapter          | Linear                   | 33     | train
-------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=1000` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

  unique_id         ds           TFT
0       0_0 2012-09-21  19426.357422
1       0_0 2012-09-28  20950.425781
2       0_0 2012-10-05  22758.187500
3       0_0 2012-10-12  24809.730469
4       0_0 2012-10-19  26718.419922
MAE on validation set: 1507.17


MAE,▁
MAE,1507.17073


In [ ]:
def weighted_mean_absolute_error(y_true, y_pred, is_holiday, weight_holiday=5):
    weights = np.where(is_holiday == 1, weight_holiday, 1)
    absolute_errors = np.abs(y_true - y_pred)
    weighted_errors = weights * absolute_errors
    wmae = np.sum(weighted_errors) / np.sum(weights)
    return wmae

In [ ]:
# Create holiday indicator in your original dataframe, convert to int (0 or 1)
df['isHoliday'] = df['IsHoliday'].astype(int)

# Prepare val_df with holiday info by merging from original df
val_df = val_df.merge(df[['unique_id', 'ds', 'isHoliday']], on=['unique_id', 'ds'], how='left')

# Now merge predictions and val_df
merged = preds.merge(val_df, on=['unique_id', 'ds'], how='left')

# Drop rows where any are missing
merged = merged.dropna(subset=['y', 'TFT', 'isHoliday'])

# Now you can calculate weighted MAE
wmae = weighted_mean_absolute_error(merged['y'], merged['TFT'], merged['isHoliday'])

print(f"Weighted MAE: {wmae:.4f}")


Weighted MAE: 1507.1707


In [ ]:
print(val_df.columns)


Index(['unique_id', 'ds', 'y'], dtype='object')
